# Test swaps

In [ ]:
from sugar import AsyncOPChainSimnet, OPChainSimnet, AsyncOPChain, AsyncBaseChain, AsyncLiskChain
from sugar.superswap import MockSuperswapRelayer, AsyncSuperswap, Superswap
from sugar.helpers import require_supersim, atime_it
from sugar.wallet import get_token_balances
from fastcore.test import test_eq

require_supersim()

In [ ]:
# check balances in simnet to make sure we have enough tokens for testing

async with AsyncOPChain() as op:
    async with AsyncOPChainSimnet() as simnet:
        tokens = await op.get_all_tokens()
        prices = await op.get_prices(tokens)
        balances = await get_token_balances(simnet, tokens, prices)
        eth = next((b for b in balances if b.token.symbol == 'ETH'), None)
        assert eth is not None, "ETH balance not found"
        test_eq(eth.balance, 10000.0)

        velo = next((b for b in balances if b.token.symbol == 'VELO'), None)
        assert velo is not None, "VELO balance not found"
        test_eq(velo.balance, 1000.0)

        usdc = next((b for b in balances if b.token.symbol == 'USDC'), None)
        assert usdc is not None, "USDC balance not found"
        test_eq(usdc.balance, 1000.0)

In [ ]:
# velo -> usdc
async with AsyncOPChain() as op:
    async with atime_it("Get quote"):
        quote = await op.get_quote(from_token=op.velo, to_token=op.usdc, amount=op.velo.parse_units(10))

    async with AsyncOPChainSimnet() as op_simnet:
        print(await op_simnet.swap_from_quote(quote, slippage=0.1))

    with OPChainSimnet() as op_simnet_sync:
        print(op_simnet_sync.swap_from_quote(quote, slippage=0.1))

# velo -> eth
async with AsyncOPChain() as op:
    async with atime_it("Get quote"):
        quote = await op.get_quote(op.velo, op.eth, op.velo.parse_units(10))

    async with AsyncOPChainSimnet() as op_simnet:
        print(await op_simnet.swap_from_quote(quote, slippage=0.1))

# eth -> velo
async with AsyncOPChain() as op:
    async with atime_it("Get quote"):
        quote = await op.get_quote(op.eth, op.velo, op.eth.parse_units(0.001))

    async with AsyncOPChainSimnet() as op_simnet:
        print(await op_simnet.swap_from_quote(quote, slippage=0.1))


Get quote took 3.1985 seconds
AttributeDict({'type': 2, 'status': 1, 'cumulativeGasUsed': 424155, 'logs': [AttributeDict({'address': '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'), HexBytes('0x000000000000000000000000f39fd6e51aad88f6f4ce6ab8827279cfffb92266'), HexBytes('0x00000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763c')], 'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), 'blockHash': HexBytes('0xdb5d4b27f37134bbc50a0037a9ce5f60e4562e5ccfe1e33b6b089a90a2da295c'), 'blockNumber': 138668989, 'blockTimestamp': '0x687bb16f', 'transactionHash': HexBytes('0x2dc2c3c0952f2edc9e41b167e5d4a31bf8d6f3b53888c62d08bc73fefa166b28'), 'transactionIndex': 0, 'logIndex': 0, 'removed': False}), AttributeDict({'address': '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('

## Superswaps

In [ ]:
# try unsupported chains

async with AsyncOPChainSimnet() as op_sim:
    from_token, to_token = AsyncOPChain.velo, AsyncBaseChain.aero 
    error = None
    try:
        tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=MockSuperswapRelayer()).swap(from_token, to_token, amount=from_token.parse_units(20))
    except ValueError as e:
        error = e
    test_eq(str(error), "Superswap only supports ['OP', 'Lisk', 'Uni']. Got OP -> Base")

# sync version
with OPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.velo, AsyncLiskChain.lsk
    tx = Superswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=from_token.parse_units(10), slippage=0.1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 1)
    last_call = relayer.get_last_call()
    test_eq(type(last_call["salt"]), str)
    test_eq(type(last_call["origin_domain"]), int)

# async version
async with AsyncOPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.velo, AsyncLiskChain.lsk
    tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=from_token.parse_units(10), slippage=0.1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 1)
    last_call = relayer.get_last_call()
    test_eq(type(last_call["salt"]), str)
    test_eq(type(last_call["origin_domain"]), int)

# to o_usdt variation
async with AsyncOPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.velo, AsyncLiskChain.o_usdt
    tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=from_token.parse_units(10), slippage=0.1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 0)

# from o_usdt variation
with OPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.o_usdt, AsyncLiskChain.lsk
    tx = Superswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=from_token.parse_units(1), slippage=0.1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 1)
    last_call = relayer.get_last_call()
    test_eq(type(last_call["salt"]), str)
    test_eq(type(last_call["origin_domain"]), int)

# async bridge
async with AsyncOPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.o_usdt, AsyncLiskChain.o_usdt
    tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=from_token.parse_units(1))
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 0)

# sync bridge
with OPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.o_usdt, AsyncLiskChain.o_usdt
    tx = Superswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=from_token.parse_units(1))
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 0)


Mock relayer received call: {'calls': [{'to': '0x0000000000000000000000001217bfe6c773eec6cc4a38b5dc45b92292b6e189', 'value': '0', 'data': '0x095ea7b300000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763cffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'}, {'to': '0x00000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763c', 'value': '0', 'data': '0x24856bc3000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000800000000000000000000000000000000000000000000000000000000000000002a1a100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000032000000000000000000000000000000000000000000000000000000000000002c00000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000